<a href="https://colab.research.google.com/github/easeandrelax/blood-sugar-level/blob/main/Welcome_To_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Data Preprocessing

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


🔹 Why Mount Google Drive?
Persistent Storage – Files in /content (Colab's default directory) get deleted after the session ends, but Google Drive keeps your files safe.

Easy File Access – Instead of re-uploading files every time, you can store them in Drive and load them whenever needed.

Handling Large Datasets – If you're working with large files (like images or CSVs), storing them in Drive prevents running out of Colab's limited storage.

Syncing with Other Devices – You can access the same files from different devices without re-downloading them.

In [2]:
import os

image_folder = "/content/drive/MyDrive/train_images"
image_files = os.listdir(image_folder)

print(f"Total images found: {len(image_files)}")
print(image_files)  # Show first 10 images


Total images found: 3
['train_images', 'train.gsheet', 'train.csv']


In [3]:
import glob

train_dir = "/content/drive/MyDrive/train_images"
image_count = len(glob.glob(train_dir + "/*.[jp][pn]g"))  # Counts JPG, JPEG, PNG
print("Number of images:", image_count)



Number of images: 0


In [4]:
image_list = !ls /content/drive/MyDrive/train_images
image_count = len(image_list)
print("Number of images:", image_count)


Number of images: 1


In [5]:
csv_path = "/content/drive/MyDrive/train_images/train.csv"
if os.path.exists(csv_path):
    print("✅ CSV file found!")
else:
    print("❌ CSV file missing. Please upload it.")


✅ CSV file found!


In [6]:
import pandas as pd

# Get list of image files
image_files = os.listdir(image_folder)

# Create a DataFrame with filenames and random labels (modify labels accordingly)
df = pd.DataFrame({"Image Name": image_files, "Label": ["class_1"] * len(image_files)})

# Save CSV in Drive
csv_path = "/content/drive/MyDrive/train_images/train.csv"
df.to_csv(csv_path, index=False)

print(f"✅ CSV file created and saved at: {csv_path}")


✅ CSV file created and saved at: /content/drive/MyDrive/train_images/train.csv


In [7]:
import pandas as pd

CSV_PATH = "/content/drive/MyDrive/train_images/train.csv"
df = pd.read_csv(CSV_PATH)

print("✅ CSV File Loaded Successfully!")
print(df.head())  # Display the first few rows


✅ CSV File Loaded Successfully!
     Image Name    Label
0  train_images  class_1
1  train.gsheet  class_1
2     train.csv  class_1


Step 1: Load and Preprocess Data

In [8]:
!ls /content/drive/MyDrive/train_images



train.csv  train.gsheet  train_images


In [9]:
!ls /content/drive/MyDrive/train_images

train.csv  train.gsheet  train_images


In [10]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Define dataset paths
dataset_path = "/content/drive/MyDrive/train_images/"
csv_path = "/content/drive/MyDrive/train_images/train.csv"  # Update with actual file path

# Load CSV
df = pd.read_csv(csv_path)
df['image_path'] = dataset_path + df['Image Name']  # Full path to images

# Show dataset preview
print(df.head())


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
     Image Name    Label                                        image_path
0  train_images  class_1  /content/drive/MyDrive/train_images/train_images
1  train.gsheet  class_1  /content/drive/MyDrive/train_images/train.gsheet
2     train.csv  class_1     /content/drive/MyDrive/train_images/train.csv


 Preprocess Images for Regression Model

In [12]:
image_path = '/content/drive/MyDrive/train_images'
IMG_SIZE = (224, 224)

# Function to load images and normalize them
def load_and_preprocess_image(image_path):
    img = load_img(image_path, target_size=IMG_SIZE)
    img_array = img_to_array(img) / 255.0  # Normalize
    return img_array

# Load all images into memory
X = np.array([load_and_preprocess_image(img) for img in df['image_path']])
y = np.array(df['Blood Sugar Level'])  # Target values

# Train-Test Split (80% Train, 20% Test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



IsADirectoryError: [Errno 21] Is a directory: '/content/drive/MyDrive/train_images/train_images'

In [13]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from PIL import UnidentifiedImageError  # Import for handling the specific error

# ... (rest of your code)
image_path = 'content/drive/MyDrive/train_images'

# Function to load images and normalize them, with error handling
def load_and_preprocess_image(image_path):
    try:
        img = load_img(image_path, target_size=IMG_SIZE)
        img_array = img_to_array(img) / 255.0  # Normalize
        return img_array
    except (FileNotFoundError, UnidentifiedImageError, OSError) as e:
        print(f"Error loading image {image_path}: {e}")
        return None  # or handle the error differently (e.g., skip the image)

# Load all images into memory, filtering out failed loads
X = np.array([img for img in [load_and_preprocess_image(img_path) for img_path in df['image_path']] if img is not None])
# Update target values to match the filtered images
valid_indices = [i for i, img in enumerate([load_and_preprocess_image(img_path) for img_path in df['image_path']]) if img is not None]
y = np.array(df['Blood Sugar Level'][valid_indices])  # Target values

# ... (rest of your code)

Error loading image /content/drive/MyDrive/train_images/train_images: [Errno 21] Is a directory: '/content/drive/MyDrive/train_images/train_images'
Error loading image /content/drive/MyDrive/train_images/train.gsheet: [Errno 95] Operation not supported: '/content/drive/MyDrive/train_images/train.gsheet'
Error loading image /content/drive/MyDrive/train_images/train.csv: cannot identify image file <_io.BytesIO object at 0x7f60fdfa5a80>
Error loading image /content/drive/MyDrive/train_images/train_images: [Errno 21] Is a directory: '/content/drive/MyDrive/train_images/train_images'
Error loading image /content/drive/MyDrive/train_images/train.gsheet: [Errno 95] Operation not supported: '/content/drive/MyDrive/train_images/train.gsheet'
Error loading image /content/drive/MyDrive/train_images/train.csv: cannot identify image file <_io.BytesIO object at 0x7f61a0b6d1c0>


KeyError: 'Blood Sugar Level'

In [14]:
import os
import glob
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from PIL import UnidentifiedImageError

# Define image size
IMG_SIZE = (224, 224)  # Resize images to 224x224

# Get all image file paths using glob
image_dir = "/content/drive/MyDrive/train_images"
image_paths = glob.glob(image_dir + "/*.[jp][pn]g")  # Matches .jpg, .jpeg, .png

# Load labels from CSV (Ensure it contains 'image_path' and 'Blood Sugar Level')
df_labels = pd.read_csv("/content/drive/MyDrive/train_images/train.csv")

# Convert to DataFrame and merge with labels
df = pd.DataFrame({"image_path": image_paths})
df = df.merge(df_labels, on="image_path", how="inner")  # Ensure only valid images are included

# Function to load and preprocess images
def load_and_preprocess_image(image_path):
    try:
        img = load_img(image_path, target_size=IMG_SIZE)  # Resize image
        img_array = img_to_array(img) / 255.0  # Normalize pixel values
        return img_array
    except (FileNotFoundError, UnidentifiedImageError, OSError) as e:
        print(f"Error loading image {image_path}: {e}")
        return None

# Load images and filter out failed loads
images = [load_and_preprocess_image(img_path) for img_path in df['image_path']]
valid_data = [(img, df['Blood Sugar Level'][i]) for i, img in enumerate(images) if img is not None]

# Convert to numpy arrays
X = np.array([img for img, _ in valid_data])
y = np.array([label for _, label in valid_data])

print(f"Loaded {len(X)} valid images out of {len(image_paths)} total.")

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Training samples: {len(X_train)}, Testing samples: {len(X_test)}")


KeyError: 'image_path'

In [15]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from PIL import UnidentifiedImageError

# Define image size
IMG_SIZE = (224, 224)

# Load the updated CSV file
csv_path = "/content/drive/MyDrive/train_images/train.csv"
df = pd.read_csv(csv_path)

# Ensure CSV has the correct columns
required_columns = {"Image_Name", "Label", "Image_Path"}
if not required_columns.issubset(df.columns):
    raise ValueError(f"CSV must contain {required_columns} columns.")

# Function to load and preprocess images
def load_and_preprocess_image(Image_Path):
    try:
        img = load_img(Image_Path, target_size=IMG_SIZE)  # Load image
        img_array = img_to_array(img) / 255.0  # Normalize
        return img_array
    except (FileNotFoundError, UnidentifiedImageError, OSError) as e:
        print(f"Error loading image {Image_Path}: {e}")
        return None

# Load images and labels
images = [load_and_preprocess_image(img_path) for img_path in df["Image_Path"]]
valid_data = [(img, label) for img, label in zip(images, df["Label"]) if img is not None]

# Convert to numpy arrays
X = np.array([img for img, _ in valid_data])
y = np.array([label for _, label in valid_data])

print(f"Loaded {len(X)} valid images out of {len(df)} total.")

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Training samples: {len(X_train)}, Testing samples: {len(X_test)}")

# Define a simple CNN model
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='linear')  # Regression for blood sugar level
])

# Compile model
model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])

# Train model
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=16)

# Save trained model
model.save("/content/drive/MyDrive/blood_sugar_model.h5")

print("Model training complete and saved successfully!")


ValueError: CSV must contain {'Label', 'Image_Name', 'Image_Path'} columns.

In [16]:
import os
print(os.path.exists('/content/drive/MyDrive/train_images/train.csv'))


True


In [19]:
import imghdr
img_path = "/content/drive/MyDrive/train_images/train_images"
print(imghdr.what(img_path))  # Should return 'png', 'jpeg', etc.



IsADirectoryError: [Errno 21] Is a directory: '/content/drive/MyDrive/train_images/train_images'

In [20]:
import os
from PIL import Image

image_dir = "/content/drive/MyDrive/train_images"
invalid_images = []

for filename in os.listdir(image_dir):
    file_path = os.path.join(image_dir, filename)
    try:
        with Image.open(file_path) as img:
            img.verify()  # Verify if the image is valid
    except Exception as e:
        invalid_images.append(file_path)

print(f"Invalid images: {len(invalid_images)}")
if invalid_images:
    print("Removing invalid images...")
    for img in invalid_images:
        os.remove(img)  # Remove corrupted images
    print("Invalid images removed.")
else:
    print("All images are valid.")


Invalid images: 3
Removing invalid images...


IsADirectoryError: [Errno 21] Is a directory: '/content/drive/MyDrive/train_images/train_images'

In [21]:
import os
from PIL import Image

def convert_images(image_dir):
    for filename in os.listdir(image_dir):
        file_path = os.path.join(image_dir, filename)
        try:
            with Image.open(file_path) as img:
                img.convert("RGB").save(file_path, "PNG")  # Save as PNG
        except Exception as e:
            print(f"Skipping {filename}: {e}")

image_dir = "/content/drive/MyDrive/train_images"
convert_images(image_dir)
print("Image conversion complete.")


Skipping train_images: [Errno 21] Is a directory: '/content/drive/MyDrive/train_images/train_images'
Skipping train.gsheet: [Errno 95] Operation not supported: '/content/drive/MyDrive/train_images/train.gsheet'
Skipping train.csv: cannot identify image file '/content/drive/MyDrive/train_images/train.csv'
Image conversion complete.


In [22]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [23]:
import pandas as pd

df = pd.read_csv("/content/drive/MyDrive/train_images/train.csv")  # Load CSV file

print(int(df["Image_Path"].isnull().sum()))  # Count missing values


KeyError: 'Image_Path'

In [24]:
import pandas as pd

# Load the CSV file
csv_path = "/content/drive/MyDrive/train_images/train.csv"
df = pd.read_csv(csv_path)

# Check for missing values in the 'Image_Path' column
missing_count = df["Image_Path"].isnull().sum()
print(f"Missing Image Paths: {int(missing_count)}")


KeyError: 'Image_Path'

In [25]:
import os

path = "/content/drive/MyDrive/train_images/"
print("Files in directory:", os.listdir(path))


Files in directory: ['train_images', 'train.gsheet', 'train.csv']


In [ ]:
import os
from PIL import Image

image_dir = "/content/drive/MyDrive/train_images/train_images"

valid_extensions = {".jpg", ".jpeg", ".png", ".bmp", ".gif"}

def convert_images(image_dir):
    for filename in os.listdir(image_dir):
        file_path = os.path.join(image_dir, filename)
        # Skip directories and non-image files
        if not os.path.isfile(file_path) or not any(filename.lower().endswith(ext) for ext in valid_extensions):
            print(f"Skipping {filename} (not an image file)")
            continue
        try:
            with Image.open(file_path) as img:
                img.convert("RGB").save(file_path, "PNG")  # Save as PNG
        except Exception as e:
            print(f"Skipping {filename}: {e}")

convert_images(image_dir)
print("Image conversion complete.")


In [ ]:
import os
import pandas as pd
import numpy as np
from PIL import Image

image_dir = "/content/drive/MyDrive/train_images/train_images"
valid_extensions = {".jpg", ".jpeg", ".png", ".bmp", ".gif"}

# List to store image data
image_data = []

# Read images and store pixel values
for filename in os.listdir(image_dir):
    file_path = os.path.join(image_dir, filename)
    if not any(filename.lower().endswith(ext) for ext in valid_extensions):
        print(f"Skipping {filename} (not an image file)")
        continue
    try:
        with Image.open(file_path) as img:
            img = img.resize((128, 128))  # Resize for uniform shape
            img_array = np.array(img)  # Convert image to NumPy array
            image_data.append(img_array.flatten())  # Flatten and store pixel values
    except Exception as e:
        print(f"Skipping {filename}: {e}")

# Convert list to Pandas DataFrame
df = pd.DataFrame(image_data)
print(df.shape)  # Check dimensions of DataFrame



In [ ]:
import os
import pandas as pd
import numpy as np
from PIL import Image, ImageFile

# Fix for truncated images
ImageFile.LOAD_TRUNCATED_IMAGES = True

image_dir = "/content/drive/MyDrive/train_images/train_images"
valid_extensions = {".jpg", ".jpeg", ".png", ".bmp", ".gif"}

image_data = []

for filename in os.listdir(image_dir):
    file_path = os.path.join(image_dir, filename)
    if not any(filename.lower().endswith(ext) for ext in valid_extensions):
        print(f"Skipping {filename} (not an image file)")
        continue
    try:
        with Image.open(file_path) as img:
            img = img.resize((128, 128))  # Resize to 128x128
            img_array = np.array(img)  # Convert to NumPy array
            image_data.append(img_array.flatten())  # Flatten and store
    except Exception as e:
        print(f"Skipping {filename}: {e}")  # Ignore errors

# Convert to Pandas DataFrame
df = pd.DataFrame(image_data)
print(f"Shape of DataFrame: {df.shape}")  # Check dimensions


In [ ]:
img = img.convert("L")  # Convert to grayscale
img_array = np.array(img).flatten()  # 128x128 = 16384 instead of 49152


In [ ]:
import matplotlib.pyplot as plt
import cv2

image_path = "your_image_path_here.jpg"
image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

if image is not None:
    # Resize image to 512x512 for faster processing
    image = cv2.resize(image, (512, 512))

    plt.imshow(image, cmap='gray')
    plt.title("Grayscale Image Check")
    plt.axis("off")
    plt.show()
else:
    print("Error: Image not found!")


In [ ]:
for filename in os.listdir(image_dir):
    file_path = os.path.join(image_dir, filename)
    print(f"Processing: {filename}")  # Debugging statement


In [ ]:
import os
print("Directory exists:", os.path.exists(image_dir))


In [ ]:
import os
print(os.listdir("/content/drive/MyDrive/train_images"))



In [1]:
import matplotlib.pyplot as plt
plt.imshow(your_image, cmap='gray')
plt.show()
()

NameError: name 'your_image' is not defined

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2

# Load an example image (modify the path based on your dataset)
image_path = "your_image_path_here.jpg"
image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # Load in grayscale mode

# Check if the image is grayscale (should have only 2 dimensions)
if len(image.shape) == 2:
    print("The image is grayscale ✅")
else:
    print("The image is NOT grayscale ❌")

# Display the image
plt.imshow(image, cmap='gray')
plt.title("Grayscale Image")
plt.axis("off")
plt.show()


AttributeError: 'NoneType' object has no attribute 'shape'

In [7]:
import cv2

image_path = "/content/drive/MyDrive/train_images/train_images.jpg"  # Change this to the correct path

image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

if image is None:
    print("Error: Image not loaded. Check the file path and format!")
else:
    print("Image loaded successfully.")
    print("Image shape:", image.shape)


Error: Image not loaded. Check the file path and format!


In [8]:
!ls /content/drive/MyDrive/train_images/


train.csv  train.gsheet  train_images


In [10]:
import os

image_path = "/content/drive/MyDrive/train_images/7005be54cab1.png"
print("Does the file exist?", os.path.exists(image_path))


Does the file exist? False


In [11]:
!ls /content/drive/MyDrive/train_images/*.png

!ls /content/drive/MyDrive/train_images/class_1/*.png
!ls /content/drive/MyDrive/train_images/class_2/*.png


ls: cannot access '/content/drive/MyDrive/train_images/*.png': No such file or directory
ls: cannot access '/content/drive/MyDrive/train_images/class_1/*.png': No such file or directory
ls: cannot access '/content/drive/MyDrive/train_images/class_2/*.png': No such file or directory


In [12]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
!ls /content/drive/MyDrive/train_images/


train.csv  train.gsheet  train_images


In [14]:
image_path = "/content/drive/MyDrive/train_images/class_1/7005be54cab1.png"
